In [3]:
! conda install -y pytorch

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/noarch::dask==2.14.0=py_0
  - defaults/noarch::numpydoc==0.9.2=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::numpy-base==1.18.1=py36hde5b4d6_1
  - defaults/noarch::s3fs==0.4.0=py_0
  - defaults/linux-64::patsy==0.5.1=py36_0
  - defaults/linux-64::scikit-image==0.16.2=py36h0573a6f_0
  - defaults/linux-64::matplotlib-base==3.1.3=py36hef1b27d_0
  - defaults/linux-64::pytables==3.6.1=py36h71ec239_0
  - defaults/linux-64::mkl_f

In [4]:
! conda install faiss-cpu -y -c pytorch

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/noarch::dask==2.14.0=py_0
  - defaults/linux-64::pytorch==1.4.0=cuda101py36h02f0884_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::numpy-base==1.18.1=py36hde5b4d6_1
  - defaults/linux-64::patsy==0.5.1=py36_0
  - defaults/linux-64::scikit-image==0.16.2=py36h0573a6f_0
  - defaults/linux-64::matplotlib-base==3.1.3=py36hef1b27d_0
  - defaults/linux-64::pytables==3.6.1=py36h71ec239_0
  - defaults/linux-64::mkl_fft==1.0.15=py36ha843

In [4]:
! conda list

# packages in environment at /home/ec2-user/anaconda3/envs/JupyterSystemEnv:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
asn1crypto                1.3.0            py36h9f0ad1d_1    conda-forge
attrs                     19.3.0                     py_0    conda-forge
autovizwidget             0.12.9                     py_0    conda-forge
awscli                    1.18.104                 pypi_0    pypi
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
bcrypt                    3.1.7                    pypi_0    pypi
bleach                    3.1.5              pyh9f0ad1d_0    conda-forge
boto3                     1.14.27                  pypi_0    pypi
botocore                  1.17.27                  pypi_0    pypi
brotlipy                  0.7.0           py36h7b6447c_1000  
ca-certificates           2020.6.20            hecda079_0    conda-forge
cached-property           1.5.

In [51]:
import sys
import numpy as np
import pandas as pd
import html.parser
import gensim
from torch.utils.tensorboard import SummaryWriter
import faiss
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

from tensorflow.python.keras.preprocessing.text import Tokenizer

In [6]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
pd.set_option('display.max_colwidth', None)

In [10]:
df = pd.read_csv("../data/Reviews.csv")

In [11]:
df.shape

(568454, 10)

In [12]:
sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2

In [13]:
import importlib

from utils.text_processing import apply_regexes, unescape_html, remove_html_tags, remove_line_breaks, preprocess

# Text Preprocessing

In [14]:
df['cleaned_text'] = df['Text'] + ' ' + df['Summary']
df['cleaned_text'].fillna("", inplace=True)

df['cleaned_text'] = df['cleaned_text'] \
    .apply(lambda x: unescape_html(remove_html_tags(remove_line_breaks(x), 'html.parser')))

In [15]:
df['preprocessed_text'] = df['cleaned_text'].apply(preprocess)

# Word To Vec

In [11]:
from nltk.tokenize import word_tokenize

review_text =  [word_tokenize(review) for review in df['preprocessed_text']]

In [12]:
review2vec = gensim.models.Word2Vec(
    review_text, min_count=10, size=100, workers=10, sg=1, iter=10, window=50
)

In [14]:
print(f'Vocab length for review2vec: {len(review2vec.wv.vocab)}')
review2vec.save("../model/review2vec.model")

Vocab length for review2vec: 26959


In [17]:
review2vec.wv.most_similar('expire')

[('expiration', 0.8689521551132202),
 ('expires', 0.8167486786842346),
 ('date', 0.8046296834945679),
 ('expired', 0.7890604734420776),
 ('expiring', 0.7793307900428772),
 ('exp', 0.7547811269760132),
 ('oct', 0.7090200185775757),
 ('august', 0.702245831489563),
 ('stale', 0.6954241991043091),
 ('expiry', 0.6935100555419922)]

## Finding Most Similar Reviews

In [15]:
def get_mean_vector(word2vec_model, words):

    if type(words) != str:
        return np.zeros(word2vec_model.wv.vector_size, )
    else:
        words = words.split()
        words = [word for word in words if word in word2vec_model.wv.vocab]
        if len(words) >= 1:
            return np.mean(word2vec_model.wv[words], axis=0)
        else:
            return np.zeros(word2vec_model.wv.vector_size, )

In [46]:
df_embed = resample(df['preprocessed_text'], replace=False, n_samples=100000)
review_text = df_embed.tolist()

review_vector = np.asarray(
    df_embed.apply(lambda x: get_mean_vector(review2vec, x)).tolist()
)

# normalize vectors
review_vector_normed = np.copy(review_vector)
nonzero_idx = np.sum(review_vector, axis=1) != 0
review_vector_normed[nonzero_idx] = review_vector[nonzero_idx] / np.linalg.norm(
    review_vector[nonzero_idx], axis=-1, keepdims=True
)

review_vector_normed = review_vector_normed.astype("float32")

In [35]:
from sklearn.utils import resample

In [48]:
"""
Finding nearest neighbor
"""
faiss_index = faiss.IndexFlatIP(review_vector_normed.shape[1])
faiss_index.add(review_vector_normed)
top_k = 5
S, I = faiss_index.search(review_vector_normed, top_k)
get_review = np.vectorize(lambda x: review_text[x])
nn_text = get_review(I)


In [62]:
df_eval = pd.DataFrame({'preprocessed_text': df_embed})

In [64]:
df_eval["nn_score"] = [score for score in S]
df_eval["nn_text"] = [text for text in nn_text]

In [69]:
df_eval.to_csv(
    "../model/review2vec_eval.csv", sep='\t', index=False
)

# Data Sampling

In [ ]:
# TODO: Move this to utility functions

In [40]:
df.loc[df['Score'] <= 3, 'ReviewSentiment'] = 0
df.loc[df['Score'] > 3, 'ReviewSentiment'] = 1

df['ReviewSentiment'] = df['ReviewSentiment'].astype(int)

## Simple Undersampling

In [41]:
negative = df[df['ReviewSentiment']==0]
positive = df[df['ReviewSentiment']==1]
print('Number of negative samples:', len(negative))
print('Number of positive samples:', len(positive))

Number of negative samples: 124677
Number of positive samples: 443777


In [42]:
positive_downsampled = resample(positive, replace=True, # sample with replacement
                                n_samples=len(negative), # match number in minority class
                                random_state=1)
print('Number of positive downsampled:', len(positive_downsampled))

downsampled = pd.concat([negative, positive_downsampled])
print('Total Number of rows after downsampling:', len(downsampled))

Number of positive downsampled: 124677
Total Number of rows after downsampling: 249354


## Train Test Split

In [43]:
x_train, x_test, y_train, y_test = train_test_split(
    downsampled['preprocessed_text'], 
    downsampled['ReviewSentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=downsampled['ReviewSentiment']
)

print('Number of train samples:', len(x_train))
print('Number of test samples:', len(x_test))

Number of train samples: 199483
Number of test samples: 49871


# Train on Review2Vec

In [17]:
try:
    review2vec
except NameError:
    review2vec = gensim.models.Word2Vec.load("../model/review2vec.model")

In [56]:
# tokenizing the reviews words

tokenizer = Tokenizer()
tokenizer.fit_on_texts(downsampled['preprocessed_text'])

# creating embedding matrix for directly feeding to Embedding() layer of Keras

num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, 100))

for word, i in tokenizer.word_index.items():
    
    if i > num_words:
        continue
    
    if word in review2vec.wv.vocab:
        embedding_vector = review2vec.wv[word]
        embedding_matrix[i] = embedding_vector

## Model

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.optimizers import Adam
from keras.initializers import Constant

Using TensorFlow backend.


In [67]:
model_review2vec_keras = Sequential()

adam = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
loss = 'categorical_crossentropy'

embedding_layer = Embedding(
    len(tokenizer.word_index)+1,
    100,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=60,
    trainable=False
)

model_review2vec_keras.add(embedding_layer)
model_review2vec_keras.add(Dropout(0.2))

model_review2vec_keras.add(LSTM(100))
model_review2vec_keras.add(Dropout(0.2))

model_review2vec_keras.add(Dense(2, activation='sigmoid'))

optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)

model_review2vec_keras.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
model_review2vec_keras.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 100)           7115400   
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 7,196,002
Trainable params: 80,602
Non-trainable params: 7,115,400
_________________________________________________________________


In [65]:
# training the model
x_train_tokenized = tokenizer.texts_to_sequences(x_train)
x_test_tokenized = tokenizer.texts_to_sequences(x_test)

In [68]:
model_review2vec_keras_trained = model_review2vec_keras.fit(
    x_train_tokenized, 
    y_train, 
    batch_size=16, #512
    epochs=10)

# TODO: will fix later

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 199483 arrays: [array([[  30],
       [  30],
       [ 750],
       [ 148],
       [   3],
       [ 125],
       [ 836],
       [ 282],
       [ 804],
       [  11],
       [   8],
       [ 255],
       [1771],
    ...

In [ ]:
model_trained_word2vec_history.evaluate(x_test_tokenized, y_test)

# Baseline With LR

In [44]:
tfidfv = TfidfVectorizer(max_df=100000)
fv_train = tfidfv.fit_transform(x_train)
fv_test = tfidfv.transform(x_test)

print('Shape of train count vector:', fv_train.shape)
print('Shape of test count vector:', fv_test.shape)

Shape of train count vector: (199483, 65118)
Shape of test count vector: (49871, 65118)


In [45]:
lr = LogisticRegression(penalty='l2', random_state=1, solver='sag', max_iter=1000, C=2)
lr.fit(fv_train, y_train)

train_acc = lr.score(fv_train, y_train)
print('Train Accuracy: %.3f' %(train_acc))

test_acc = lr.score(fv_test, y_test)
print('Test Accuracy: %0.3f' % (test_acc))

Train Accuracy: 0.908
Test Accuracy: 0.889


# Train on Review2Vec With Mean Vectors (By Review)

In [83]:
vocab = review2vec.wv
w2v_model = review2vec
w2v_dim = w2v_model.wv.vector_size

In [81]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models.keyedvectors import BaseKeyedVectors

In [74]:
from typing import Optional

In [75]:
def get_gensim_word_index(kv:BaseKeyedVectors, oov_token:Optional[str]=None, num_words:Optional[int]=None):
    """ Get word index from gensim keyed vectors so word indices are sorted by frequency.
    Modified from https://github.com/keras-team/keras-preprocessing/blob/d7225f269ec28737d1fda036c2d26277b5863f87/keras_preprocessing/text.py#L199
    :return: Two dictionaries of word to index and index to word.
    :param kv: gensim Keyed Vectors
    :param oov_token: Optional. A Special string to represent out-of-vocabulary token, e.g. <unk>.
    :param num_words: Optioanl. Number of most frequent tokens to keep. If None, keep all.
    """
    word_counts = {}
    vocab = kv.vocab
    for w in vocab:
        word_counts[w] = vocab[w].count
    wcounts = list(word_counts.items())
    wcounts.sort(key=lambda x: x[1], reverse=True)
    # first token is the out-of-vocabulary token if specified
    if oov_token is None:
        sorted_voc = []
    else:
        sorted_voc = [oov_token]
    # keep top frequent tokens
    if num_words is not None:
        wcounts = wcounts[:num_words]
    sorted_voc.extend(wc[0] for wc in wcounts)

    # note that index 0 is reserved, never assigned to an existing word
    # Fixme: index 0 can be assigned to oov token
    word_index = dict(
        zip(sorted_voc, list(range(1, len(sorted_voc) + 1))))

    index_word = {c: w for w, c in word_index.items()}

    return word_index, index_word

In [76]:
class PreTrainedTextTokenizer(Tokenizer):
    """Use a pretrained gensim keyed vector as text tokenizer, which transforms list of strings to list of indices"""
    
    def fit_on_pretrained_embedding(self, gensim_kv: dict):
        """Wrapper for :func:`get_gensim_word_index`. Providing 2 attributes needed for the Base class
        :param gensim_kv:
        """
        self.word_index, self.index_word = get_gensim_word_index(
            gensim_kv,
            oov_token=self.oov_token,
            num_words=self.num_words
        )

In [77]:
def text_tokenizer_fit(df, gensim_kv: dict=None, num_words=None):
    """
    :param df: input DataFrame
    :param gensim_kv: pretrained gensim keyedvectors
    :return: dict of text tokenizers
    """
    text_tokenizer = dict()
    
    filters = '!"#$%&()*_+,-./:;<=>?@[\]^`{|}~'
    lower = True

    tokenizer = PreTrainedTextTokenizer(filters=filters, lower=lower, num_words=num_words)
    tokenizer.fit_on_pretrained_embedding(gensim_kv=gensim_kv)
    
    return tokenizer

In [82]:
def text_tokenizer_transform(df, tokenizer, max_len=6):
    """
    Transform all features
    :return: tuple of (cat features, cat embed features, text features)
    """
    text_padded = dict()
    
    text_sequences = tokenizer.texts_to_sequences(df['preprocessed_text'])
    # if not passed any value, then set default maxlen=6

    text_padded = pad_sequences(text_sequences, maxlen=max_len,
                                padding='post', truncating='post')
    return text_padded

In [ ]:
df_train = ?

text_tokenizer = text_tokenizer_fit(df, gensim_kv=vocab)

# featurization
text_padded = text_tokenizer_transform(df_train, text_tokenizer)
text_dims = text_padded.shape[1]

# use text_padded as `x_train`
x_train = ?
y_train = ?




In [ ]:
print('test')

In [ ]:
loss = kwargs.pop("loss", "default")

input_layers, output_layers = concatenate_features(
    text_cols, text_tokenizer, text_dims, w2v_model, w2v_dim,
)


model = build_mlp_model(self.model_type, cat_features_dim, self.cat_embed_cols,
                                 self.text_cols, self.encoders['text'], text_dims,
                                 self.w2v_model, self.w2v_dim,
                                 num_classes,
                                 cat_embed_dims=cat_embed_dims,
                                 cat_embed_embed_dims=cat_embed_embed_dims,
                                 units_list = units_list,
                                 loss=loss,
                                 user2vec_model=self.user2vec_model,
                                 user2vec_model_dim=self.user2vec_model_dim,
                                 userid_mapping=userid_mapping,
                                 learning_rate=learning_rate)

In [ ]:
def build_text_embed_layer(text_col, tokenizer, text_dim, embed_dim,
                           text_w2v=None, pre_trained=False):
    """
    :param text_col: text column name to be embeded (string)
    :param tokenizer: keras tokenizer for the column
    :param text_dim: length of the padded text input (e.g. maxlen=6, then this value is 6)
    :param embed_dim: embedding dimension
    :param text_w2v: pre-trained word2vec model
    :param pre_trained: train the embedding E2E if False; else use the pre-trained w2v model
    :return: input & output layer
    """
    num_word = len(tokenizer.word_index)
    embedding_matrix = np.zeros((num_word+1, embed_dim))
    embed_input = Input(shape=(text_dim,), name=text_col+'_input')
    if pre_trained:
        kv = text_w2v.wv
        for word, i in tokenizer.word_index.items():
            if word in kv.vocab:
                embedding_matrix[i] = kv[word]
        # Todo: remove these. Not valid anymore
        # n_no_match = np.sum(np.sum(embedding_matrix, axis=1) == 0)
        # print(f'Null word embeddings: {n_no_match}, {round(n_no_match/num_word*100,2)}% of total unique words' )
        embed_output = Embedding(embedding_matrix.shape[0],  # or len(word_index) + 1
                              embedding_matrix.shape[1],  # or EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=text_dim,
                              trainable=False,
                              name=text_col+'_embedding')(embed_input)
    else:
        embed_output = Embedding(embedding_matrix.shape[0],
                              embedding_matrix.shape[1],
                              input_length=text_dim,
                              trainable=True,
                              name=text_col+'_embedding')(embed_input)
    embed_output = Flatten()(embed_output)
    embed_output = Dropout(0.2)(embed_output)
    return embed_input, embed_output

# Custom Embedding

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

In [123]:
x_train, x_test, y_train, y_test = train_test_split(
    downsampled['preprocessed_text'], 
    downsampled['ReviewSentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=downsampled['ReviewSentiment']
)

print('Number of train samples:', len(x_train))
print('Number of test samples:', len(x_test))



Number of train samples: 199483
Number of test samples: 49871


In [131]:
tfidfv = TfidfVectorizer(max_df=100000)
fv_train = tfidfv.fit_transform(x_train)
fv_test = tfidfv.transform(x_test)

print('Shape of train count vector:', fv_train.shape)
print('Shape of test count vector:', fv_test.shape)

Shape of train count vector: (199483, 65118)
Shape of test count vector: (49871, 65118)


In [132]:
lr = LogisticRegression(penalty='l2', random_state=1, solver='sag', max_iter=1000, C=2)
lr.fit(fv_train, y_train)

train_acc = lr.score(fv_train, y_train)
print('Train Accuracy: %.3f' %(train_acc))

test_acc = lr.score(fv_test, y_test)
print('Test Accuracy: %0.3f' % (test_acc))

Train Accuracy: 0.908
Test Accuracy: 0.889
